In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.interpolate import interp1d, RBFInterpolator
from scipy.spatial.distance import pdist
from scipy.ndimage import gaussian_filter

# **1단계: 데이터 로드**
fitting_df = pd.read_csv("fittings_filled.csv")
V_values = fitting_df["V"].values
t0_values = fitting_df["t0"].values
mu_values = fitting_df["mu"].values * 1e8  # cm² → μm² 변환

Vefft0_values = (V_values - 1.1) * t0_values
Vefft0mu_values = Vefft0_values * mu_values

# valid_mask_t0 = ~np.isnan(t0_values)
# valid_mask_mu = ~np.isnan(mu_values)

# t0_interp = interp1d(V_values[valid_mask_t0], t0_values[valid_mask_t0], kind="linear", fill_value="extrapolate")
# mu_interp = interp1d(V_values[valid_mask_mu], mu_values[valid_mask_mu], kind="linear", fill_value="extrapolate")

# t0_values[np.isnan(t0_values)] = t0_interp(V_values[np.isnan(t0_values)])
# mu_values[np.isnan(mu_values)] = mu_interp(V_values[np.isnan(mu_values)])

# **2단계: 데이터 생성**
L = 400  
Vt_grid = np.linspace(3000, 17000, 1000)

# t_grid = np.logspace(np.log10(300), np.log10(6000), 200)

V_unique = np.unique(V_values)  # 보간 없이 원래 V 값만 사용

V_list, t_list, Lzsq_list = [], [], []
Vt_list = []

Vefft_f_values = []  # 최종 도달 시간 저장

for V, t0, mu in zip(V_values, t0_values, mu_values):
    t_f = t0 + (L**2) / (2 * V * mu)  # t_f 계산
    Vefft_f_values.append((V-1.1)*t_f)  # 리스트에 저장

    Lzsq_values = L**2 + 2 * V * mu * t0 - 2 * mu * Vt_grid
    Lzsq_values = np.clip(Lzsq_values, 0, 160000)
    
    # z_values = L - np.sqrt(np.maximum(L**2 - 2 * V * mu * (t_grid - t0), 0))
    # z_values = np.clip(z_values, 0, 400)

    V_list.extend([V] * len(Vt_grid))
    Vt_list.extend(Vt_grid)
    Lzsq_list.extend(Lzsq_values)

# **3단계: 중복 데이터 제거**
data = np.array([Vt_list, V_list, Lzsq_list]).T
unique_data = np.unique(data, axis=0)  
Vt_list, V_list, Lzsq_list = unique_data[:, 0], unique_data[:, 1], unique_data[:, 2]

# **4단계: 중복 제거 후 데이터 저장**
export_df = pd.DataFrame({"V_eff*Time (s)": Vt_list, "Voltage (V)": V_list, "z (μm)": Lzsq_list})
export_df.to_csv("Newcolormap_data.csv", index=False)

# **6단계: RBF 보간**
distances = pdist(np.array([Vt_list, V_list]).T)
mean_dist = np.mean(distances)  

interp_func = RBFInterpolator(
    np.array([Vt_list, V_list]).T, 
    Lzsq_list, 
    kernel="multiquadric",  
    epsilon=mean_dist  # 데이터 기반 epsilon 설정
)

# **7단계: 보간된 Grid 생성 (V 값 보간 없이 원래 값만 사용)**
VT_grid, V_grid = np.meshgrid(Vt_grid, V_unique)
LZsq_grid = interp_func(np.array([VT_grid.ravel(), V_grid.ravel()]).T).reshape(VT_grid.shape)

# **8단계: Gaussian 필터 적용**
LZsq_grid = np.clip(LZsq_grid, 0, 160000)
LZsq_grid = gaussian_filter(Z_grid, sigma=1.0)

# **9단계: 컬러맵 설정**
cmap_main = plt.get_cmap("inferno")  # 기존 컬러맵
cmap_colors = ["black"] + [cmap_main(i) for i in np.linspace(0, 1, 256)]
custom_cmap = colors.ListedColormap(cmap_colors)

# **10단계: 그래프 출력**
plt.figure(figsize=(8, 6))
c = plt.contourf(VT_grid, V_grid, LZsq_grid, levels=100, cmap='plasma')

# **11단계: 로그 스케일, 컬러바, t0 및 t_f 값 추가**
plt.yscale("log")
plt.colorbar(c, label="z (μm)")
plt.scatter(Vefft0_values, V_values, color="white", marker="o", label="Veff t0 values")  # 흰색 원
plt.scatter(Vefft_f_values, V_values, color="cyan", marker="x", label="Veff t_f values")  # 파란색 X

plt.xlabel("Time (s)")
plt.ylabel("Voltage (V)")
plt.title("Fitted Charge Transport Model with t0 and t_f")
plt.xlim(3000, 17000)
plt.legend()
plt.show()
